In [2]:
import pandas as pd
import os
import pickle
from collections import defaultdict
from itertools import combinations
import time
from pathlib import Path
import csv
import networkx as nx

In [3]:
# Configuração do backend do matplotlib para notebooks
import os
# Remove a variável de ambiente que causa conflito
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']

import matplotlib
matplotlib.use('Agg')  # Backend não-interativo que funciona em qualquer ambiente
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Para garantir que os gráficos sejam exibidos no notebook
plt.ion()  # Ativa modo interativo

# Carregando os dados

In [4]:
root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
# garante que encontramos o dataset independentemente do cwd do notebook
input_file = root / 'datasets' / 'netflix_titles.csv'

print(f"Lendo: {input_file}")

df = pd.read_csv(input_file)

Lendo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/datasets/netflix_titles.csv


## Análise geral

In [5]:
# Etapa 2: Parse do CSV e construção das estruturas de co-atores

actor_coactors = defaultdict(set)
pair_counts = defaultdict(int)
actor_appearances = defaultdict(int)

for idx, cast in enumerate(df['cast'].fillna('')):
    if not cast:
        continue
    actors = [a.strip() for a in cast.split(',') if a.strip()]
    for a in actors:
        actor_appearances[a] += 1
    for a, b in combinations(actors, 2):
        key = tuple(sorted((a, b)))
        pair_counts[key] += 1
        actor_coactors[a].add(b)
        actor_coactors[b].add(a)

num_actors_with_coactors = len(actor_coactors)
num_actors_with_appearances = len(actor_appearances)
num_pairs = len(pair_counts)

print(f"Total de linhas no CSV: {df.shape[0]}")
print(f"Atores com pelo menos 1 co-ator: {num_actors_with_coactors}")
print(f"Atores com aparições registradas: {num_actors_with_appearances}")
print(f"Pares únicos de atores (potenciais arestas): {num_pairs}")

# Tops para inspeção
top_by_degree = sorted(((actor, len(neigh)) for actor, neigh in actor_coactors.items()), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por grau (número de coatores):')
for actor, deg in top_by_degree:
    print(f"{actor}: {deg}")

top_by_appearances = sorted(actor_appearances.items(), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por aparições:')
for actor, cnt in top_by_appearances:
    print(f"{actor}: {cnt}")

# Salvando intermediários em results/q1
out_dir = root / 'results'
out_dir.mkdir(parents=True, exist_ok=True)

# Salvar tops em CSVs para inspeção
pd.DataFrame(top_by_appearances, columns=['Actor', 'Appearances']).to_csv(out_dir / 'top_by_appearances.csv', index=False)
pd.DataFrame(top_by_degree, columns=['Actor', 'Degree']).to_csv(out_dir / 'top_by_degree.csv', index=False)

end = time.time()
print(f"\nArquivos intermediários salvos em: {out_dir}")


Total de linhas no CSV: 8807
Atores com pelo menos 1 co-ator: 36039
Atores com aparições registradas: 36439
Pares únicos de atores (potenciais arestas): 289207

Top 20 por grau (número de coatores):
Anupam Kher: 273
Samuel L. Jackson: 239
Takahiro Sakurai: 228
Fred Tatasciore: 226
Yuichi Nakamura: 223
Yuki Kaji: 220
Shah Rukh Khan: 210
Fred Armisen: 209
Akshay Kumar: 193
Katsuyuki Konishi: 191
Jun Fukuyama: 188
Om Puri: 187
Junichi Suwabe: 185
Naseeruddin Shah: 183
Boman Irani: 183
Hiroshi Kamiya: 182
James Franco: 182
Maya Rudolph: 181
Paresh Rawal: 179
Amitabh Bachchan: 178

Top 20 por aparições:
Anupam Kher: 43
Shah Rukh Khan: 35
Julie Tejwani: 33
Naseeruddin Shah: 32
Takahiro Sakurai: 32
Rupa Bhimani: 31
Akshay Kumar: 30
Om Puri: 30
Yuki Kaji: 29
Amitabh Bachchan: 28
Paresh Rawal: 28
Boman Irani: 27
Rajesh Kava: 26
Vincent Tong: 26
Andrea Libman: 25
Kareena Kapoor: 25
Samuel L. Jackson: 24
John Cleese: 24
Jigna Bhardwaj: 23
Fred Tatasciore: 23

Arquivos intermediários salvos em: /h

## Grafo geral

In [6]:
# Montagem do grafo de coautoria geral usando networkx

# Obter todos os atores únicos
all_actors = set(actor_appearances.keys())

print(f"Total de atores únicos: {len(all_actors)}")
print(f"Total de pares de coatores: {len(pair_counts)}")

# Construir grafo ponderado (undirected)
G = nx.Graph()
G.add_nodes_from(all_actors)

for (a, b), w in pair_counts.items():
    # adicionar aresta com atributo weight (número de vezes que atuaram juntos)
    G.add_edge(a, b, weight=w)

n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
print(f"\nGrafo de coautoria: nós={n_nodes}, arestas={n_edges}")

# Salvando o grafo em formato GraphML para análise posterior
geral_dir = out_dir / 'geral'
geral_dir.mkdir(parents=True, exist_ok=True)
graphml_path = geral_dir / 'grafo_coautoria.graphml'
nx.write_graphml(G, graphml_path)
print(f"Grafo salvo em: {graphml_path}")

Total de atores únicos: 36439
Total de pares de coatores: 289207

Grafo de coautoria: nós=36439, arestas=289207
Grafo salvo em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/geral/grafo_coautoria.graphml


## Medidas de centralidade geral

In [7]:
# Cálculo das centralidades e métricas globais do grafo de coautoria
# Com sistema de cache - calcula, normaliza e salva incrementalmente

print("Calculando métricas globais...")

# Métricas globais
density = nx.density(G)
components = list(nx.connected_components(G))
num_components = len(components)
largest_cc = max(components, key=len) if components else set()
largest_cc_size = len(largest_cc)

print(f"Densidade: {density:.6g}")
print(f"Número de nós: {n_nodes}")
print(f"Número de arestas: {n_edges}")
print(f"Componentes: {num_components}; maior componente: {largest_cc_size} nós")

# Salvar métricas globais
global_metrics = pd.DataFrame({
    'Metric': ['Nodes', 'Edges', 'Density', 'Components', 'Largest_Component_Size'],
    'Value': [n_nodes, n_edges, density, num_components, largest_cc_size]
})
global_metrics.to_csv(geral_dir / 'global_metrics.csv', index=False)
print(f"✓ Métricas globais salvas em: global_metrics.csv")

# ========================================================================
# Função auxiliar para normalizar valores
# ========================================================================
def normalize_dict(values_dict):
    """Normaliza valores de um dicionário para o intervalo [0, 1]"""
    values = list(values_dict.values())
    min_val = min(values)
    max_val = max(values)
    if max_val == min_val:
        return {k: 0.0 for k in values_dict.keys()}
    return {k: (v - min_val) / (max_val - min_val) for k, v in values_dict.items()}

# ========================================================================
# Cálculo incremental de centralidades com cache (valores normalizados)
# ========================================================================

# Inicializar DataFrame base com os atores
actors_list = list(G.nodes())
metrics_df = pd.DataFrame({'Actor': actors_list})

# Degree raw (sempre calcula, é rápido)
print('\n[1/5] Calculando Degree...')
degree_raw = dict(G.degree())
degree_normalized = normalize_dict(degree_raw)
metrics_df['Degree'] = [degree_normalized.get(n, 0) for n in actors_list]
print(f"✓ Degree calculado e normalizado para {len(actors_list)} atores")

# ========================================================================
# 2. Degree Centrality (já vem normalizado do NetworkX)
# ========================================================================
degree_cache_file = geral_dir / 'cache_degree_centrality.pkl'
if degree_cache_file.exists():
    print('\n[2/5] Carregando Degree Centrality do cache...')
    with open(degree_cache_file, 'rb') as f:
        degree_c = pickle.load(f)
    print(f"✓ Degree Centrality carregado do cache (já normalizado)")
else:
    print('\n[2/5] Calculando Degree Centrality...')
    degree_c = nx.degree_centrality(G)
    # Degree centrality já vem normalizado do NetworkX
    # Salvar cache
    with open(degree_cache_file, 'wb') as f:
        pickle.dump(degree_c, f)
    print(f"✓ Degree Centrality calculado e salvo em cache (já normalizado)")

metrics_df['DegreeCentrality'] = [degree_c.get(n, 0) for n in actors_list]

# ========================================================================
# 3. Closeness Centrality
# ========================================================================
closeness_cache_file = geral_dir / 'cache_closeness_centrality.pkl'
if closeness_cache_file.exists():
    print('\n[3/5] Carregando Closeness Centrality do cache...')
    with open(closeness_cache_file, 'rb') as f:
        closeness_c = pickle.load(f)
    print(f"✓ Closeness Centrality carregado do cache (já normalizado)")
else:
    print('\n[3/5] Calculando Closeness Centrality...')
    closeness_c_raw = nx.closeness_centrality(G)
    # Normalizar antes de salvar
    closeness_c = normalize_dict(closeness_c_raw)
    # Salvar cache normalizado
    with open(closeness_cache_file, 'wb') as f:
        pickle.dump(closeness_c, f)
    print(f"✓ Closeness Centrality calculado, normalizado e salvo em cache")

metrics_df['Closeness'] = [closeness_c.get(n, 0) for n in actors_list]

# ========================================================================
# 5. PageRank
# ========================================================================
pagerank_cache_file = geral_dir / 'cache_pagerank.pkl'
if pagerank_cache_file.exists():
    print('\n[4/5] Carregando PageRank do cache...')
    with open(pagerank_cache_file, 'rb') as f:
        pagerank = pickle.load(f)
    print(f"✓ PageRank carregado do cache (já normalizado)")
else:
    print('\n[4/5] Calculando PageRank...')
    pagerank_raw = nx.pagerank(G, alpha=0.85, max_iter=100)
    # Normalizar antes de salvar
    pagerank = normalize_dict(pagerank_raw)
    # Salvar cache normalizado
    with open(pagerank_cache_file, 'wb') as f:
        pickle.dump(pagerank, f)
    print(f"✓ PageRank calculado, normalizado e salvo em cache")

metrics_df['PageRank'] = [pagerank.get(n, 0) for n in actors_list]

# ========================================================================
# 3. Betweenness Centrality
# ========================================================================
betweenness_cache_file = geral_dir / 'cache_betweenness_centrality.pkl'
if betweenness_cache_file.exists():
    print('\n[5/5] Carregando Betweenness Centrality do cache...')
    with open(betweenness_cache_file, 'rb') as f:
        betweenness_c = pickle.load(f)
    print(f"✓ Betweenness Centrality carregado do cache (já normalizado)")
else:
    print('\n[5/5] Calculando Betweenness Centrality... (pode demorar)')
    betweenness_c_raw = nx.betweenness_centrality(G)
    # Normalizar antes de salvar
    betweenness_c = normalize_dict(betweenness_c_raw)
    # Salvar cache normalizado
    with open(betweenness_cache_file, 'wb') as f:
        pickle.dump(betweenness_c, f)
    print(f"✓ Betweenness Centrality calculado, normalizado e salvo em cache")

metrics_df['Betweenness'] = [betweenness_c.get(n, 0) for n in actors_list]


# ========================================================================
# Salvar resultados finais (já normalizados)
# ========================================================================
print('\nSalvando resultados finais...')

# CSV com todas as métricas de centralidade (já normalizadas)
metrics_csv = geral_dir / 'graph_metrics.csv'
metrics_df.to_csv(metrics_csv, index=False)
print(f"✓ Todas as centralidades (normalizadas): {metrics_csv.name}")

# CSV com top 10 de cada medida (valores já normalizados)
top10_degree = metrics_df.sort_values('Degree', ascending=False).head(10)[['Actor', 'Degree']]
top10_betweenness = metrics_df.sort_values('Betweenness', ascending=False).head(10)[['Actor', 'Betweenness']]
top10_closeness = metrics_df.sort_values('Closeness', ascending=False).head(10)[['Actor', 'Closeness']]
top10_pagerank = metrics_df.sort_values('PageRank', ascending=False).head(10)[['Actor', 'PageRank']]

top10_degree.to_csv(geral_dir / 'top10_degree.csv', index=False)
top10_betweenness.to_csv(geral_dir / 'top10_betweenness.csv', index=False)
top10_closeness.to_csv(geral_dir / 'top10_closeness.csv', index=False)
top10_pagerank.to_csv(geral_dir / 'top10_pagerank.csv', index=False)

print(f"✓ Top 10 por Degree (normalizado): top10_degree.csv")
print(f"✓ Top 10 por Betweenness (normalizado): top10_betweenness.csv")
print(f"✓ Top 10 por Closeness (normalizado): top10_closeness.csv")
print(f"✓ Top 10 por PageRank (normalizado): top10_pagerank.csv")

print(f"\n{'='*60}")
print(f"Todos os arquivos salvos em: {geral_dir}")
print(f"{'='*60}")

# Mostrar top 10 por cada medida
print('\n' + '='*60)
print('Top 10 por Degree (normalizado):')
print('='*60)
print(top10_degree)

print('\n' + '='*60)
print('Top 10 por Betweenness (normalizado):')
print('='*60)
print(top10_betweenness)

print('\n' + '='*60)
print('Top 10 por Closeness (normalizado):')
print('='*60)
print(top10_closeness)

print('\n' + '='*60)
print('Top 10 por PageRank (normalizado):')
print('='*60)
print(top10_pagerank)

print('\n' + '='*60)
print('Métricas Globais:')
print('='*60)
print(global_metrics)

Calculando métricas globais...
Densidade: 0.00043563
Número de nós: 36439
Número de arestas: 289207
Componentes: 874; maior componente: 32935 nós
✓ Métricas globais salvas em: global_metrics.csv

[1/5] Calculando Degree...
✓ Degree calculado e normalizado para 36439 atores

[2/5] Carregando Degree Centrality do cache...
✓ Degree Centrality carregado do cache (já normalizado)

[3/5] Calculando Closeness Centrality...
✓ Closeness Centrality calculado, normalizado e salvo em cache

[4/5] Calculando PageRank...
✓ PageRank calculado, normalizado e salvo em cache

[5/5] Calculando Betweenness Centrality... (pode demorar)
✓ Betweenness Centrality calculado, normalizado e salvo em cache

Salvando resultados finais...
✓ Todas as centralidades (normalizadas): graph_metrics.csv
✓ Top 10 por Degree (normalizado): top10_degree.csv
✓ Top 10 por Betweenness (normalizado): top10_betweenness.csv
✓ Top 10 por Closeness (normalizado): top10_closeness.csv
✓ Top 10 por PageRank (normalizado): top10_pageran

## Detecção de comunidades geral

In [ ]:
# Detecção de comunidades

# Grafo de 100 coautores

In [ ]:
# Etapa 3 (separada): Selecionar TOP_N atores por grau (conexões)
# Não executo esta célula automaticamente — confirme quando quiser rodar.

# Parâmetro dinâmico: altere antes de executar, se desejar
TOP_N = 100

# Determina paths conforme o notebook

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results' / 'q1'
out_dir.mkdir(parents=True, exist_ok=True)

# Tenta usar variáveis em memória (caso a célula de parsing tenha sido executada);
# caso contrário, carrega os pickles gerados pela etapa de parsing.
try:
    actor_coactors
    pair_counts
    actor_appearances
except NameError:
    # tenta carregar arquivos em results/q1
    try:
        with open(out_dir / 'actor_coactors.pkl', 'rb') as f:
            actor_coactors = pickle.load(f)
        with open(out_dir / 'pair_counts.pkl', 'rb') as f:
            pair_counts = pickle.load(f)
        with open(out_dir / 'actor_appearances.pkl', 'rb') as f:
            actor_appearances = pickle.load(f)
        print('Carreguei pickles de', out_dir)
    except FileNotFoundError:
        raise FileNotFoundError('Estruturas de parsing não encontradas em memória nem em results/q1. Execute a célula de parsing primeiro.')

# Calcula grau (número de coatores) e seleciona TOP_N
degrees = {actor: len(neigh) for actor, neigh in actor_coactors.items()}
all_actors_count = len(degrees)
sorted_by_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
top_actors = sorted_by_degree[:TOP_N]
actual_top_n = len(top_actors)
print(f"Atores disponíveis: {all_actors_count}, selecionando TOP_N = {TOP_N} => selecionados: {actual_top_n}")

top_names = [a for a, _ in top_actors]
top_set = set(top_names)

# Filtrar pares onde ambos estão no top
filtered_pairs = {pair: cnt for pair, cnt in pair_counts.items() if pair[0] in top_set and pair[1] in top_set}
num_filtered_pairs = len(filtered_pairs)
print(f"Pares no subgrafo dos TOP {actual_top_n}: {num_filtered_pairs}")

# --- Usando pandas para gerar os CSVs (mais simples e robusto) ---
# links dataframe
links_rows = [(a, b, cnt) for (a, b), cnt in filtered_pairs.items()]
links_df = pd.DataFrame(links_rows, columns=['Source', 'Target', 'Movies_Count'])
links_csv = out_dir / 'links_top.csv'
links_df.to_csv(links_csv, index=False)

# top actors dataframe
top_rows = [(a, degrees.get(a, 0), actor_appearances.get(a, 0)) for a, _ in top_actors]
top_df = pd.DataFrame(top_rows, columns=['Actor', 'Degree', 'Appearances'])
top_csv = out_dir / 'top_actors.csv'
top_df.to_csv(top_csv, index=False)

# Salvar pickles úteis
with open(out_dir / 'filtered_pair_counts.pkl', 'wb') as f:
    pickle.dump(filtered_pairs, f)
with open(out_dir / 'top_actor_names.pkl', 'wb') as f:
    pickle.dump(top_names, f)

print(f"Arquivos gerados (não para Flourish ainda):\n- {links_csv}\n- {top_csv}\n- {out_dir / 'filtered_pair_counts.pkl'}\n- {out_dir / 'top_actor_names.pkl'}")

# Resumo rápido dos 10 atores com maior grau selecionados
print('\nTop 10 (dos selecionados) por grau:')
for actor, deg in top_actors[:10]:
    print(f"{actor}: grau={deg}, aparições={actor_appearances.get(actor, 0)}")


Atores disponíveis: 36039, selecionando TOP_N = 100 => selecionados: 100
Pares no subgrafo dos TOP 100: 583
Arquivos gerados (não para Flourish ainda):
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/links_top.csv
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/top_actors.csv
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/filtered_pair_counts.pkl
- /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/q1/top_actor_names.pkl

Top 10 (dos selecionados) por grau:
Anupam Kher: grau=273, aparições=43
Samuel L. Jackson: grau=239, aparições=24
Takahiro Sakurai: grau=228, aparições=32
Fred Tatasciore: grau=226, aparições=23
Yuichi Nakamura: grau=223, aparições=19
Yuki Kaji: grau=220, aparições=29
Shah Rukh Khan: grau=210, aparições=35
Fred Armisen:

In [ ]:
# Etapa 4: Construir grafo NetworkX e calcular métricas (densidade, centralidades, PageRank)
# Esta célula roda a análise do grafo a partir de `filtered_pair_counts.pkl` e `top_actor_names.pkl`.

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results'
out_dir.mkdir(parents=True, exist_ok=True)


print(f"Nomes no top: {len(top_names)}; pares filtrados: {len(filtered_pairs)}")

# Construir grafo ponderado (undirected)
G = nx.Graph()
G.add_nodes_from(top_names)
for (a, b), w in filtered_pairs.items():
    # adicionar aresta com atributo weight
    G.add_edge(a, b, weight=w)

n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
print(f"Grafo: nós={n_nodes}, arestas={n_edges}")

# Métricas globais
density = nx.density(G)
components = list(nx.connected_components(G))
num_components = len(components)
largest_cc = max(components, key=len) if components else set()
largest_cc_size = len(largest_cc)

print(f"Densidade: {density:.6g}")
print(f"Componentes: {num_components}; maior componente: {largest_cc_size} nós")

# Centralidades
print('\nCalculando centralidades... (isso pode levar algum tempo para betweenness se exato)')
# Degree centrality (normalizada)
degree_c = nx.degree_centrality(G)
# Betweenness centrality (exata ou aproximada)
betweenness_c = nx.betweenness_centrality(G)
# Closeness centrality
closeness_c = nx.closeness_centrality(G)
# PageRank (usa weights)
pagerank = nx.pagerank(G, alpha=0.85, max_iter=100)

# Degree raw (número de vizinhos)
degree_raw = dict(G.degree())

# Montar DataFrame com métricas
metrics_df = pd.DataFrame({
    'Actor': list(G.nodes()),
    'Degree': [degree_raw.get(n, 0) for n in G.nodes()],
    'DegreeCentrality': [degree_c.get(n, 0) for n in G.nodes()],
    'Betweenness': [betweenness_c.get(n, 0) for n in G.nodes()],
    'Closeness': [closeness_c.get(n, 0) for n in G.nodes()],
    'PageRank': [pagerank.get(n, 0) for n in G.nodes()],
})

# Normalizar colunas (opcional) — apenas como colunas separadas para inspeção
metrics_df['Degree_norm'] = (metrics_df['Degree'] - metrics_df['Degree'].min()) / (metrics_df['Degree'].max() - metrics_df['Degree'].min())
metrics_df['Betweenness_norm'] = (metrics_df['Betweenness'] - metrics_df['Betweenness'].min()) / (metrics_df['Betweenness'].max() - metrics_df['Betweenness'].min())
metrics_df['Closeness_norm'] = (metrics_df['Closeness'] - metrics_df['Closeness'].min()) / (metrics_df['Closeness'].max() - metrics_df['Closeness'].min())
metrics_df['PageRank_norm'] = (metrics_df['PageRank'] - metrics_df['PageRank'].min()) / (metrics_df['PageRank'].max() - metrics_df['PageRank'].min())

# Salvar resultados
metrics_csv = out_dir / 'graph_metrics.csv'
metrics_df.to_csv(metrics_csv, index=False)
# salvar grafo em gpickle


# Mostrar top 10 por cada medida
print(f"\nMétricas salvas em: {metrics_csv}")

print('\nTop 10 por Degree:')
print(metrics_df.sort_values('Degree', ascending=False).head(10)[['Actor','Degree']])
print('\nTop 10 por Betweenness:')
print(metrics_df.sort_values('Betweenness', ascending=False).head(10)[['Actor','Betweenness']])
print('\nTop 10 por Closeness:')
print(metrics_df.sort_values('Closeness', ascending=False).head(10)[['Actor','Closeness']])
print('\nTop 10 por PageRank:')
print(metrics_df.sort_values('PageRank', ascending=False).head(10)[['Actor','PageRank']])

# Guardar as variáveis no notebook para uso futuro
G_graph = G
filtered_pairs_graph = filtered_pairs
metrics = metrics_df

print('\nEtapa 4 concluída — valide os resultados antes de prosseguirmos para as centralidades mais custosas (se necessário) ou detecção de comunidades.')


Nomes no top: 100; pares filtrados: 583
Grafo: nós=100, arestas=583
Densidade: 0.117778
Componentes: 1; maior componente: 100 nós

Calculando centralidades... (isso pode levar algum tempo para betweenness se exato)

Métricas salvas em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/graph_metrics.csv

Top 10 por Degree:
               Actor  Degree
4    Yuichi Nakamura      27
2   Takahiro Sakurai      26
5          Yuki Kaji      25
10      Jun Fukuyama      25
25       Daisuke Ono      24
75  Nobuhiko Okamoto      23
43     Kana Hanazawa      23
42         Ai Kayano      23
50  Miyuki Sawashiro      22
12    Junichi Suwabe      22

Top 10 por Betweenness:
                 Actor  Betweenness
30       Kari Wahlgren     0.417440
0          Anupam Kher     0.173614
87  Chloë Grace Moretz     0.126796
83       Gerard Butler     0.096958
89       Paul Giamatti     0.095672
27     Elizabeth Banks     0.070178
3      Fred Tatasciore

In [ ]:
# Etapa 6: Louvain + Geração de arquivos para Flourish
# Detecta comunidades com Louvain e gera links.csv e points.csv para visualização no Flourish

from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.quality import modularity
import pandas as pd
import pickle
from pathlib import Path

# Parâmetros ajustáveis
LOUVAIN_RESOLUTION = 1  # padrão 1.0; aumentar => mais comunidades
USE_LARGEST_CC = True
LOUVAIN_SEED = 42  # seed para reprodutibilidade

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir = root / 'results' / 'grafo_coautoria'
out_dir.mkdir(parents=True, exist_ok=True)

# Carregar grafo se necessário
try:
    G
    filtered_pairs
except NameError:
    raise FileNotFoundError('Grafo não encontrado. Execute a célula que constrói o grafo (Etapa 4) primeiro.')

# Subgrafo onde rodar Louvain
if USE_LARGEST_CC:
    comps = list(nx.connected_components(G))
    largest = max(comps, key=len)
    G_louvain = G.subgraph(largest).copy()
else:
    G_louvain = G

print(f"Rodando Louvain (NetworkX nativo) em n={G_louvain.number_of_nodes()} nós, m={G_louvain.number_of_edges()} arestas")
print(f"Parâmetro de resolução: {LOUVAIN_RESOLUTION}, seed: {LOUVAIN_SEED}")

# Executar Louvain (retorna lista de sets/frozensets)
communities_sets = louvain_communities(G_louvain, weight='weight', resolution=LOUVAIN_RESOLUTION, seed=LOUVAIN_SEED)

# Converter para lista de listas
communities_list = [list(c) for c in communities_sets]

# Calcular modularidade
mod = modularity(G_louvain, communities_list, weight='weight')
num_communities = len(communities_list)

print(f"\nModularidade obtida: {mod:.3f}")
print(f"Número de comunidades: {num_communities}")

# Estatísticas das comunidades
sizes = pd.Series([len(c) for c in communities_list])
print(f"\nDistribuição de tamanhos (min={sizes.min()}, max={sizes.max()}, mean={sizes.mean():.1f}, median={sizes.median():.1f})")

# Criar dicionário node -> community_id
partition = {}
for cid, comm in enumerate(communities_list):
    for node in comm:
        partition[node] = cid

# Estender partição para todos os nós do grafo original
comm_dict_louvain = {}
for node in G.nodes():
    if node in partition:
        comm_dict_louvain[node] = partition[node]
    else:
        comm_dict_louvain[node] = -1  # Nós fora do maior componente

print("\n" + "="*80)
print("GERANDO ARQUIVOS PARA FLOURISH")
print("="*80)

# ========== 1. GERAR links.csv ==========
# Filtrar apenas arestas do subgrafo Louvain
links_data = []
for (a, b), weight in filtered_pairs.items():
    # Incluir apenas se ambos estão no G_louvain
    if a in G_louvain.nodes() and b in G_louvain.nodes():
        links_data.append((a, b, weight))

links_df = pd.DataFrame(links_data, columns=['Source', 'Target', 'Value'])
links_csv = out_dir / 'links.csv'
links_df.to_csv(links_csv, index=False)

print(f"\n✅ links.csv gerado:")
print(f"   - {len(links_df)} arestas")
print(f"   - Arquivo: {links_csv}")

# ========== 2. GERAR points.csv ==========
# Calcular métricas de centralidade para o subgrafo Louvain
betweenness_louvain = nx.betweenness_centrality(G_louvain, weight='weight')
closeness_louvain = nx.closeness_centrality(G_louvain)
pagerank_louvain = nx.pagerank(G_louvain, weight='weight')

# Normalizar métricas (min-max normalization)
betweenness_values = list(betweenness_louvain.values())
closeness_values = list(closeness_louvain.values())
pagerank_values = list(pagerank_louvain.values())

betw_min, betw_max = min(betweenness_values), max(betweenness_values)
clos_min, clos_max = min(closeness_values), max(closeness_values)
pr_min, pr_max = min(pagerank_values), max(pagerank_values)

points_data = []
for actor in G_louvain.nodes():
    # Normalizar valores
    betw_norm = (betweenness_louvain[actor] - betw_min) / (betw_max - betw_min) if betw_max > betw_min else 0
    clos_norm = (closeness_louvain[actor] - clos_min) / (clos_max - clos_min) if clos_max > clos_min else 0
    pr_norm = (pagerank_louvain[actor] - pr_min) / (pr_max - pr_min) if pr_max > pr_min else 0
    
    points_data.append((
        actor,  # id
        comm_dict_louvain[actor],  # Community
        G_louvain.degree(actor),  # Degree (número de conexões)
        actor_appearances.get(actor, 0),  # Appearances (total de filmes)
        round(betw_norm, 4),  # Betweenness (normalizado)
        round(clos_norm, 4),  # Closeness (normalizado)
        round(pr_norm, 4)  # PageRank (normalizado)
    ))

points_df = pd.DataFrame(points_data, columns=['id', 'Community', 'Degree', 'Appearances', 'Betweenness', 'Closeness', 'PageRank'])
points_csv = out_dir / 'points.csv'
points_df.to_csv(points_csv, index=False)

print(f"\n✅ points.csv gerado:")
print(f"   - {len(points_df)} atores")
print(f"   - {points_df['Community'].nunique()} comunidades")
print(f"   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank")
print(f"   - Arquivo: {points_csv}")

coauthors_density = nx.density(G_louvain)

# ========== ANÁLISE: TOP 10 POR CENTRALIDADE ==========
print("\n" + "="*80)
print("TOP 10 ATORES POR MEDIDA DE CENTRALIDADE")
print("="*80)

print("\n🔗 Top 10 por Degree (número de conexões):")
for idx, row in points_df.nlargest(10, 'Degree').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Degree']} conexões (filmes: {row['Appearances']})")

print("\n🌉 Top 10 por Betweenness (intermediação):")
for idx, row in points_df.nlargest(10, 'Betweenness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Betweenness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n📍 Top 10 por Closeness (proximidade):")
for idx, row in points_df.nlargest(10, 'Closeness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Closeness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n⭐ Top 10 por PageRank (importância):")
for idx, row in points_df.nlargest(10, 'PageRank').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['PageRank']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n\n >>> Densidade do grafo de coautoria analisado: {:.6g} <<<".format(coauthors_density))

# ========== 3. SALVAR METADADOS ==========
# Salvar stats de Louvain
louvain_stats = {
    'resolution': LOUVAIN_RESOLUTION,
    'seed': LOUVAIN_SEED,
    'modularity': mod,
    'num_communities': num_communities,
    'nodes_analyzed': G_louvain.number_of_nodes(),
    'edges_analyzed': G_louvain.number_of_edges()
}

pd.DataFrame([louvain_stats]).to_csv(out_dir / 'louvain_stats.csv', index=False)

# Salvar partição em pickle
with open(out_dir / 'communities_louvain.pkl', 'wb') as f:
    pickle.dump(comm_dict_louvain, f)

# ========== 4. RESUMO DAS COMUNIDADES ==========
print("\n" + "="*80)
print("RESUMO DAS COMUNIDADES")
print("="*80)

comm_sizes = points_df['Community'].value_counts().sort_values(ascending=False)
print(f"\nTop 10 maiores comunidades (ID: tamanho):")
for idx, (comm_id, size) in enumerate(comm_sizes.head(10).items(), 1):
    print(f"  {idx}. Comunidade {comm_id}: {size} atores")

print("\n" + "="*80)
print("ARQUIVOS FLOURISH PRONTOS!")
print("="*80)
print(f"\n📁 Diretório: {out_dir}")
print(f"   ├── links.csv    ({len(links_df)} arestas)")
print(f"   ├── points.csv   ({len(points_df)} nós, {num_communities} comunidades)")
print(f"   ├── louvain_stats.csv")
print(f"   └── communities_louvain.pkl")
print("\n✨ Importe links.csv e points.csv no Flourish Network Graph!")
print("   - Use 'id' como node identifier")
print("   - Use 'Community' para colorir os nós")

Rodando Louvain (NetworkX nativo) em n=100 nós, m=583 arestas
Parâmetro de resolução: 1, seed: 42

Modularidade obtida: 0.507
Número de comunidades: 3

Distribuição de tamanhos (min=12, max=61, mean=33.3, median=27.0)

GERANDO ARQUIVOS PARA FLOURISH

✅ links.csv gerado:
   - 583 arestas
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_coautoria/links.csv



✅ points.csv gerado:
   - 100 atores
   - 3 comunidades
   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_coautoria/points.csv

TOP 10 ATORES POR MEDIDA DE CENTRALIDADE

🔗 Top 10 por Degree (número de conexões):
   5. Yuichi Nakamura: 27 conexões (filmes: 19)
   3. Takahiro Sakurai: 26 conexões (filmes: 32)
   6. Yuki Kaji: 25 conexões (filmes: 29)
   11. Jun Fukuyama: 25 conexões (filmes: 16)
   26. Daisuke Ono: 24 conexões (filmes: 22)
   43. Ai Kayano: 23 conexões (filmes: 19)
   44. Kana Hanazawa: 23 conexões (filmes: 18)
   76. Nobuhiko Okamoto: 23 conexões (filmes: 12)
   10. Katsuyuki Konishi: 22 conexões (filmes: 12)
   13. Junichi Suwabe: 22 conexões (filmes: 21)

🌉 Top 10 por Betweenness (intermediação):
   31. Kari Wahlgren: 1.0000 (grau: 18, filmes: 16)
   88. Chloë Grace Moretz: 0.3857 (grau: 13, filmes: 9)
   1. 

# Atores com mais filmes

In [ ]:
# Etapa 7: Selecionar TOP_N atores por APARIÇÕES (número de filmes)
# Critério diferente do grafo de coautoria (que usa grau/conexões)

# Parâmetro dinâmico: altere antes de executar, se desejar
TOP_N_APPEARANCES = 100

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()

# Validar que temos os dados necessários
try:
    actor_coactors
    pair_counts
    actor_appearances
except NameError:
    raise FileNotFoundError('Estruturas de parsing não encontradas. Execute a célula de parsing (Etapa 2) primeiro.')

# Selecionar TOP_N por número de aparições (filmes)
sorted_by_appearances = sorted(actor_appearances.items(), key=lambda x: x[1], reverse=True)
top_actors_appearances = sorted_by_appearances[:TOP_N_APPEARANCES]
actual_top_n_appearances = len(top_actors_appearances)

print(f"Atores disponíveis: {len(actor_appearances)}, selecionando TOP_N_APPEARANCES = {TOP_N_APPEARANCES} => selecionados: {actual_top_n_appearances}")

top_names_appearances = [a for a, _ in top_actors_appearances]
top_set_appearances = set(top_names_appearances)

# Filtrar pares onde ambos estão no top (por aparições)
filtered_pairs_appearances = {pair: cnt for pair, cnt in pair_counts.items() 
                              if pair[0] in top_set_appearances and pair[1] in top_set_appearances}
num_filtered_pairs_appearances = len(filtered_pairs_appearances)

print(f"Pares no subgrafo dos TOP {actual_top_n_appearances} (por aparições): {num_filtered_pairs_appearances}")

# Resumo rápido dos 10 atores com mais filmes selecionados
print(f'\nTop 10 (dos selecionados) por aparições:')
for actor, apps in top_actors_appearances[:10]:
    # Calcular grau (conexões) para comparação
    degree = len(actor_coactors.get(actor, set()))
    print(f"{actor}: {apps} filmes, grau={degree}")

print(f'\n✅ Seleção por aparições concluída. Próximo: construir grafo e calcular métricas.')

Atores disponíveis: 36439, selecionando TOP_N_APPEARANCES = 100 => selecionados: 100
Pares no subgrafo dos TOP 100 (por aparições): 460

Top 10 (dos selecionados) por aparições:
Anupam Kher: 43 filmes, grau=273
Shah Rukh Khan: 35 filmes, grau=210
Julie Tejwani: 33 filmes, grau=37
Naseeruddin Shah: 32 filmes, grau=183
Takahiro Sakurai: 32 filmes, grau=228
Rupa Bhimani: 31 filmes, grau=28
Akshay Kumar: 30 filmes, grau=193
Om Puri: 30 filmes, grau=187
Yuki Kaji: 29 filmes, grau=220
Amitabh Bachchan: 28 filmes, grau=178

✅ Seleção por aparições concluída. Próximo: construir grafo e calcular métricas.


In [ ]:
# Etapa 8: Construir grafo NetworkX e calcular métricas (Grafo por Aparições)
# Mesma análise do grafo de coautoria, mas com atores selecionados por número de filmes

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_appearances = root / 'results' / 'grafo_atores_mais_filmes'
out_dir_appearances.mkdir(parents=True, exist_ok=True)

print(f"Nomes no top (por aparições): {len(top_names_appearances)}; pares filtrados: {len(filtered_pairs_appearances)}")

# Construir grafo ponderado (undirected)
G_appearances = nx.Graph()
G_appearances.add_nodes_from(top_names_appearances)
for (a, b), w in filtered_pairs_appearances.items():
    # adicionar aresta com atributo weight
    G_appearances.add_edge(a, b, weight=w)

n_nodes_app = G_appearances.number_of_nodes()
n_edges_app = G_appearances.number_of_edges()
print(f"Grafo: nós={n_nodes_app}, arestas={n_edges_app}")

# Métricas globais
density_app = nx.density(G_appearances)
components_app = list(nx.connected_components(G_appearances))
num_components_app = len(components_app)
largest_cc_app = max(components_app, key=len) if components_app else set()
largest_cc_size_app = len(largest_cc_app)

print(f"Densidade: {density_app:.6g}")
print(f"Componentes: {num_components_app}; maior componente: {largest_cc_size_app} nós")

# Centralidades
print('\nCalculando centralidades... (isso pode levar algum tempo para betweenness se exato)')
# Degree centrality (normalizada)
degree_c_app = nx.degree_centrality(G_appearances)
# Betweenness centrality (exata ou aproximada)
betweenness_c_app = nx.betweenness_centrality(G_appearances)
# Closeness centrality
closeness_c_app = nx.closeness_centrality(G_appearances)
# PageRank (usa weights)
pagerank_app = nx.pagerank(G_appearances, alpha=0.85, max_iter=100)

# Degree raw (número de vizinhos)
degree_raw_app = dict(G_appearances.degree())

# Montar DataFrame com métricas
metrics_df_app = pd.DataFrame({
    'Actor': list(G_appearances.nodes()),
    'Degree': [degree_raw_app.get(n, 0) for n in G_appearances.nodes()],
    'DegreeCentrality': [degree_c_app.get(n, 0) for n in G_appearances.nodes()],
    'Betweenness': [betweenness_c_app.get(n, 0) for n in G_appearances.nodes()],
    'Closeness': [closeness_c_app.get(n, 0) for n in G_appearances.nodes()],
    'PageRank': [pagerank_app.get(n, 0) for n in G_appearances.nodes()],
})

# Normalizar colunas (opcional) — apenas como colunas separadas para inspeção
metrics_df_app['Degree_norm'] = (metrics_df_app['Degree'] - metrics_df_app['Degree'].min()) / (metrics_df_app['Degree'].max() - metrics_df_app['Degree'].min())
metrics_df_app['Betweenness_norm'] = (metrics_df_app['Betweenness'] - metrics_df_app['Betweenness'].min()) / (metrics_df_app['Betweenness'].max() - metrics_df_app['Betweenness'].min())
metrics_df_app['Closeness_norm'] = (metrics_df_app['Closeness'] - metrics_df_app['Closeness'].min()) / (metrics_df_app['Closeness'].max() - metrics_df_app['Closeness'].min())
metrics_df_app['PageRank_norm'] = (metrics_df_app['PageRank'] - metrics_df_app['PageRank'].min()) / (metrics_df_app['PageRank'].max() - metrics_df_app['PageRank'].min())

# Salvar resultados
metrics_csv_app = out_dir_appearances / 'graph_metrics.csv'
metrics_df_app.to_csv(metrics_csv_app, index=False)

# Mostrar top 10 por cada medida
print(f"\nMétricas salvas em: {metrics_csv_app}")

print('\nTop 10 por Degree:')
print(metrics_df_app.sort_values('Degree', ascending=False).head(10)[['Actor','Degree']])
print('\nTop 10 por Betweenness:')
print(metrics_df_app.sort_values('Betweenness', ascending=False).head(10)[['Actor','Betweenness']])
print('\nTop 10 por Closeness:')
print(metrics_df_app.sort_values('Closeness', ascending=False).head(10)[['Actor','Closeness']])
print('\nTop 10 por PageRank:')
print(metrics_df_app.sort_values('PageRank', ascending=False).head(10)[['Actor','PageRank']])

print('\n✅ Etapa 8 concluída — métricas calculadas para o grafo de atores com mais filmes.')

Nomes no top (por aparições): 100; pares filtrados: 460
Grafo: nós=100, arestas=460
Densidade: 0.0929293
Componentes: 9; maior componente: 82 nós

Calculando centralidades... (isso pode levar algum tempo para betweenness se exato)

Métricas salvas em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_atores_mais_filmes/graph_metrics.csv

Top 10 por Degree:
               Actor  Degree
0        Anupam Kher      26
11       Boman Irani      24
7            Om Puri      22
1     Shah Rukh Khan      22
15    Kareena Kapoor      21
52      Rajpal Yadav      21
59     Jackie Shroff      20
9   Amitabh Bachchan      20
10      Paresh Rawal      19
48       Anil Kapoor      19

Top 10 por Betweenness:
              Actor  Betweenness
0       Anupam Kher     0.300174
90     Willem Dafoe     0.173630
19  Fred Tatasciore     0.147288
97      Keith David     0.145079
74    Alfred Molina     0.137690
66    Kari Wahlgren     0.112241
53

In [ ]:
# Etapa 9: Louvain + Geração de arquivos para Flourish (Grafo por Aparições)
# Detecta comunidades com Louvain e gera links.csv e points.csv para visualização no Flourish

# Parâmetros ajustáveis
LOUVAIN_RESOLUTION_APP = 1  # padrão 1.0; aumentar => mais comunidades
USE_LARGEST_CC_APP = False
LOUVAIN_SEED_APP = 42  # seed para reprodutibilidade

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_app = root / 'results' / 'grafo_atores_mais_filmes'
out_dir_app.mkdir(parents=True, exist_ok=True)

# Carregar grafo se necessário
try:
    G_appearances
    filtered_pairs_appearances
except NameError:
    raise FileNotFoundError('Grafo não encontrado. Execute a célula que constrói o grafo (Etapa 8) primeiro.')

# Subgrafo onde rodar Louvain
if USE_LARGEST_CC_APP:
    comps_app = list(nx.connected_components(G_appearances))
    largest_app = max(comps_app, key=len)
    G_louvain_app = G_appearances.subgraph(largest_app).copy()
else:
    G_louvain_app = G_appearances

print(f"Rodando Louvain (NetworkX nativo) em n={G_louvain_app.number_of_nodes()} nós, m={G_louvain_app.number_of_edges()} arestas")
print(f"Parâmetro de resolução: {LOUVAIN_RESOLUTION_APP}, seed: {LOUVAIN_SEED_APP}")

# Calcular densidade do subgrafo Louvain
density_louvain_app = nx.density(G_louvain_app)

# Executar Louvain (retorna lista de sets/frozensets)
communities_sets_app = louvain_communities(G_louvain_app, weight='weight', resolution=LOUVAIN_RESOLUTION_APP, seed=LOUVAIN_SEED_APP)

# Converter para lista de listas
communities_list_app = [list(c) for c in communities_sets_app]

# Calcular modularidade
mod_app = modularity(G_louvain_app, communities_list_app, weight='weight')
num_communities_app = len(communities_list_app)

print(f"\nModularidade obtida: {mod_app:.3f}")
print(f"Número de comunidades: {num_communities_app}")

# Estatísticas das comunidades
sizes_app = pd.Series([len(c) for c in communities_list_app])
print(f"\nDistribuição de tamanhos (min={sizes_app.min()}, max={sizes_app.max()}, mean={sizes_app.mean():.1f}, median={sizes_app.median():.1f})")

# Criar dicionário node -> community_id
partition_app = {}
for cid, comm in enumerate(communities_list_app):
    for node in comm:
        partition_app[node] = cid

# Estender partição para todos os nós do grafo original
comm_dict_louvain_app = {}
for node in G_appearances.nodes():
    if node in partition_app:
        comm_dict_louvain_app[node] = partition_app[node]
    else:
        comm_dict_louvain_app[node] = -1  # Nós fora do maior componente

print("\n" + "="*80)
print("GERANDO ARQUIVOS PARA FLOURISH (ATORES COM MAIS FILMES)")
print("="*80)

# ========== 1. GERAR links.csv ==========
# Filtrar apenas arestas do subgrafo Louvain
links_data_app = []
for (a, b), weight in filtered_pairs_appearances.items():
    # Incluir apenas se ambos estão no G_louvain_app
    if a in G_louvain_app.nodes() and b in G_louvain_app.nodes():
        links_data_app.append((a, b, weight))

links_df_app = pd.DataFrame(links_data_app, columns=['Source', 'Target', 'Value'])
links_csv_app = out_dir_app / 'links.csv'
links_df_app.to_csv(links_csv_app, index=False)

print(f"\n✅ links.csv gerado:")
print(f"   - {len(links_df_app)} arestas")
print(f"   - Arquivo: {links_csv_app}")

# ========== 2. GERAR points.csv ==========
# Calcular métricas de centralidade para o subgrafo Louvain
betweenness_louvain_app = nx.betweenness_centrality(G_louvain_app, weight='weight')
closeness_louvain_app = nx.closeness_centrality(G_louvain_app)
pagerank_louvain_app = nx.pagerank(G_louvain_app, weight='weight')

# Normalizar métricas (min-max normalization)
betweenness_values_app = list(betweenness_louvain_app.values())
closeness_values_app = list(closeness_louvain_app.values())
pagerank_values_app = list(pagerank_louvain_app.values())

betw_min_app, betw_max_app = min(betweenness_values_app), max(betweenness_values_app)
clos_min_app, clos_max_app = min(closeness_values_app), max(closeness_values_app)
pr_min_app, pr_max_app = min(pagerank_values_app), max(pagerank_values_app)

points_data_app = []
for actor in G_louvain_app.nodes():
    # Normalizar valores
    betw_norm_app = (betweenness_louvain_app[actor] - betw_min_app) / (betw_max_app - betw_min_app) if betw_max_app > betw_min_app else 0
    clos_norm_app = (closeness_louvain_app[actor] - clos_min_app) / (clos_max_app - clos_min_app) if clos_max_app > clos_min_app else 0
    pr_norm_app = (pagerank_louvain_app[actor] - pr_min_app) / (pr_max_app - pr_min_app) if pr_max_app > pr_min_app else 0
    
    points_data_app.append((
        actor,  # id
        comm_dict_louvain_app[actor],  # Community
        G_louvain_app.degree(actor),  # Degree (número de conexões)
        actor_appearances.get(actor, 0),  # Appearances (total de filmes)
        round(betw_norm_app, 4),  # Betweenness (normalizado)
        round(clos_norm_app, 4),  # Closeness (normalizado)
        round(pr_norm_app, 4)  # PageRank (normalizado)
    ))

points_df_app = pd.DataFrame(points_data_app, columns=['id', 'Community', 'Degree', 'Appearances', 'Betweenness', 'Closeness', 'PageRank'])
points_csv_app = out_dir_app / 'points.csv'
points_df_app.to_csv(points_csv_app, index=False)

print(f"\n✅ points.csv gerado:")
print(f"   - {len(points_df_app)} atores")
print(f"   - {points_df_app['Community'].nunique()} comunidades")
print(f"   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank")
print(f"   - Arquivo: {points_csv_app}")

    # ========== ANÁLISE: TOP 10 POR CENTRALIDADE ==========
print("\n" + "="*80)
print("TOP 10 ATORES POR MEDIDA DE CENTRALIDADE")
print("="*80)

print("\n🔗 Top 10 por Degree (número de conexões):")
for idx, row in points_df_app.nlargest(10, 'Degree').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Degree']} conexões (filmes: {row['Appearances']})")

print("\n🌉 Top 10 por Betweenness (intermediação):")
for idx, row in points_df_app.nlargest(10, 'Betweenness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Betweenness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n📍 Top 10 por Closeness (proximidade):")
for idx, row in points_df_app.nlargest(10, 'Closeness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Closeness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n⭐ Top 10 por PageRank (importância):")
for idx, row in points_df_app.nlargest(10, 'PageRank').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['PageRank']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n\n >>> Densidade do grafo de atores com mais filmes analisado: {:.6g} <<<".format(density_louvain_app))

# ========== 3. SALVAR METADADOS ==========
# Salvar stats de Louvain
louvain_stats_app = {
    'resolution': LOUVAIN_RESOLUTION_APP,
    'seed': LOUVAIN_SEED_APP,
    'modularity': mod_app,
    'num_communities': num_communities_app,
    'nodes_analyzed': G_louvain_app.number_of_nodes(),
    'edges_analyzed': G_louvain_app.number_of_edges()
}

pd.DataFrame([louvain_stats_app]).to_csv(out_dir_app / 'louvain_stats.csv', index=False)

# Salvar partição em pickle
with open(out_dir_app / 'communities_louvain.pkl', 'wb') as f:
    pickle.dump(comm_dict_louvain_app, f)

# ========== 4. ANÁLISE DETALHADA DAS COMUNIDADES ==========
print("\n" + "="*80)
print("ANÁLISE DETALHADA DAS COMUNIDADES")
print("="*80)

# Estatísticas por comunidade
community_stats = []
for comm_id in sorted(set(partition_app.values())):
    members = [actor for actor in G_louvain_app.nodes() if partition_app.get(actor) == comm_id]
    
    # Métricas da comunidade
    total_actors = len(members)
    total_appearances = sum(actor_appearances.get(actor, 0) for actor in members)
    avg_appearances = total_appearances / total_actors if total_actors > 0 else 0
    
    # Ator mais prolífico da comunidade
    top_actor = max(members, key=lambda a: actor_appearances.get(a, 0))
    top_actor_apps = actor_appearances.get(top_actor, 0)
    
    community_stats.append({
        'Community_ID': comm_id,
        'Size': total_actors,
        'Total_Appearances': total_appearances,
        'Avg_Appearances': avg_appearances,
        'Top_Actor': top_actor,
        'Top_Actor_Appearances': top_actor_apps
    })

community_stats_df = pd.DataFrame(community_stats)
community_stats_df.to_csv(out_dir_app / 'community_analysis.csv', index=False)

# Exibir resumo
print(f"\nTop 10 maiores comunidades:")
for idx, row in community_stats_df.nlargest(13, 'Size').iterrows():
    print(f"  Comunidade {row['Community_ID']}: {row['Size']} atores | "
          f"Média: {row['Avg_Appearances']:.1f} filmes | "
          f"Top: {row['Top_Actor']} ({row['Top_Actor_Appearances']} filmes)")

print("\n" + "="*80)
print("ARQUIVOS FLOURISH PRONTOS!")
print("="*80)
print(f"\n📁 Diretório: {out_dir_app}")
print(f"   ├── links.csv              ({len(links_df_app)} arestas)")
print(f"   ├── points.csv             ({len(points_df_app)} nós, {num_communities_app} comunidades)")
print(f"   ├── graph_metrics.csv      (métricas NetworkX)")
print(f"   ├── louvain_stats.csv      (estatísticas Louvain)")
print(f"   ├── community_analysis.csv (análise detalhada por comunidade)")
print(f"   └── communities_louvain.pkl")
print("\n✨ Importe links.csv e points.csv no Flourish Network Graph!")
print("   - Use 'id' como node identifier")
print("   - Use 'Community' para colorir os nós")
print("   - Use 'Appearances' para tamanho dos nós")

Rodando Louvain (NetworkX nativo) em n=100 nós, m=460 arestas
Parâmetro de resolução: 1, seed: 42

Modularidade obtida: 0.763
Número de comunidades: 13

Distribuição de tamanhos (min=1, max=32, mean=7.7, median=4.0)

GERANDO ARQUIVOS PARA FLOURISH (ATORES COM MAIS FILMES)

✅ links.csv gerado:
   - 460 arestas
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_atores_mais_filmes/links.csv



✅ points.csv gerado:
   - 100 atores
   - 13 comunidades
   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_atores_mais_filmes/points.csv

TOP 10 ATORES POR MEDIDA DE CENTRALIDADE

🔗 Top 10 por Degree (número de conexões):
   1. Anupam Kher: 26 conexões (filmes: 43)
   12. Boman Irani: 24 conexões (filmes: 27)
   2. Shah Rukh Khan: 22 conexões (filmes: 35)
   8. Om Puri: 22 conexões (filmes: 30)
   16. Kareena Kapoor: 21 conexões (filmes: 25)
   53. Rajpal Yadav: 21 conexões (filmes: 17)
   10. Amitabh Bachchan: 20 conexões (filmes: 28)
   60. Jackie Shroff: 20 conexões (filmes: 17)
   7. Akshay Kumar: 19 conexões (filmes: 30)
   11. Paresh Rawal: 19 conexões (filmes: 28)

🌉 Top 10 por Betweenness (intermediação):
   1. Anupam Kher: 1.0000 (grau: 26, filmes: 43)
   91. Willem Dafoe: 0.6109 (grau: 5, filmes: 15)
   67. Kari Wahl

# Diretores com mais filmes

In [ ]:
# Etapa 10: Parse do CSV e construção das estruturas de co-diretores

director_codirectors = defaultdict(set)
pair_counts_directors = defaultdict(int)
director_appearances = defaultdict(int)

for idx, directors_cell in enumerate(df['director'].fillna('')):
    if not directors_cell:
        continue
    directors = [d.strip() for d in str(directors_cell).split(',') if d.strip()]
    for d in directors:
        director_appearances[d] += 1
    for d1, d2 in combinations(directors, 2):
        key = tuple(sorted((d1, d2)))
        pair_counts_directors[key] += 1
        director_codirectors[d1].add(d2)
        director_codirectors[d2].add(d1)

num_directors_with_codirectors = len(director_codirectors)
num_directors_with_appearances = len(director_appearances)
num_pairs_directors = len(pair_counts_directors)

print(f"Total de linhas no CSV: {df.shape[0]}")
print(f"Diretores com pelo menos 1 co-diretor: {num_directors_with_codirectors}")
print(f"Diretores com aparições registradas: {num_directors_with_appearances}")
print(f"Pares únicos de diretores (potenciais arestas): {num_pairs_directors}")

# Tops para inspeção
top_by_degree_dir = sorted(((director, len(neigh)) for director, neigh in director_codirectors.items()), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por grau (número de co-diretores):')
for director, deg in top_by_degree_dir:
    print(f"{director}: {deg}")

top_by_appearances_dir = sorted(director_appearances.items(), key=lambda x: x[1], reverse=True)[:20]
print('\nTop 20 por aparições:')
for director, cnt in top_by_appearances_dir:
    print(f"{director}: {cnt}")

print(f"\n✅ Parse de diretores concluído.")

Total de linhas no CSV: 8807
Diretores com pelo menos 1 co-diretor: 1196
Diretores com aparições registradas: 4993
Pares únicos de diretores (potenciais arestas): 1307

Top 20 por grau (número de co-diretores):
Roger Allers: 22
Chris Buck: 14
Byron Howard: 13
Peter Farrelly: 12
Shinji Aramaki: 12
Rusty Cundieff: 12
Jennifer Lee: 12
Patrick Osborne: 12
Lauren MacMullan: 12
John Kahrs: 12
Nathan Greno: 12
Stevie Wermers: 12
Dean Wellins: 12
Kevin Deters: 12
Mike Gabriel: 12
Mark Henn: 12
Gautham Vasudev Menon: 11
Mamoru Oshii: 11
Hideki Futamura: 11
Toshiyuki Kanno: 11

Top 20 por aparições:
Rajiv Chilaka: 22
Jan Suter: 21
Raúl Campos: 19
Suhas Kadav: 16
Marcus Raboy: 16
Jay Karas: 15
Cathy Garcia-Molina: 13
Youssef Chahine: 12
Martin Scorsese: 12
Jay Chapman: 12
Steven Spielberg: 11
Don Michael Paul: 10
David Dhawan: 9
Yılmaz Erdoğan: 9
Anurag Kashyap: 9
Shannon Hartman: 9
Quentin Tarantino: 8
Robert Rodriguez: 8
Hakan Algül: 8
Hanung Bramantyo: 8

✅ Parse de diretores concluído.


In [ ]:
# Etapa 11: Selecionar TOP_N diretores por grau (co-direções)

# Parâmetro dinâmico: altere antes de executar, se desejar
TOP_N_DIRECTORS = 100

# Validar que temos os dados necessários
try:
    director_codirectors
    pair_counts_directors
    director_appearances
except NameError:
    raise FileNotFoundError('Estruturas de parsing de diretores não encontradas. Execute a célula de parse (Etapa 10) primeiro.')

# Calcula grau (número de co-diretores) e seleciona TOP_N
degrees_directors = {director: len(neigh) for director, neigh in director_codirectors.items()}
all_directors_count = len(degrees_directors)
sorted_by_degree_dir = sorted(degrees_directors.items(), key=lambda x: x[1], reverse=True)
top_directors = sorted_by_degree_dir[:TOP_N_DIRECTORS]
actual_top_n_dir = len(top_directors)

print(f"Diretores disponíveis: {all_directors_count}, selecionando TOP_N_DIRECTORS = {TOP_N_DIRECTORS} => selecionados: {actual_top_n_dir}")

top_names_directors = [d for d, _ in top_directors]
top_set_directors = set(top_names_directors)

# Filtrar pares onde ambos estão no top
filtered_pairs_directors = {pair: cnt for pair, cnt in pair_counts_directors.items() 
                            if pair[0] in top_set_directors and pair[1] in top_set_directors}
num_filtered_pairs_dir = len(filtered_pairs_directors)

print(f"Pares no subgrafo dos TOP {actual_top_n_dir}: {num_filtered_pairs_dir}")

# Resumo rápido dos 10 diretores com maior grau selecionados
print(f'\nTop 10 (dos selecionados) por grau:')
for director, deg in top_directors[:10]:
    print(f"{director}: grau={deg}, aparições={director_appearances.get(director, 0)}")

print(f'\n✅ Seleção por grau (co-direções) concluída. Próximo: construir grafo e calcular métricas.')

Diretores disponíveis: 1196, selecionando TOP_N_DIRECTORS = 100 => selecionados: 100
Pares no subgrafo dos TOP 100: 495

Top 10 (dos selecionados) por grau:
Roger Allers: grau=22, aparições=3
Chris Buck: grau=14, aparições=3
Byron Howard: grau=13, aparições=2
Peter Farrelly: grau=12, aparições=4
Shinji Aramaki: grau=12, aparições=2
Rusty Cundieff: grau=12, aparições=2
Jennifer Lee: grau=12, aparições=1
Patrick Osborne: grau=12, aparições=1
Lauren MacMullan: grau=12, aparições=1
John Kahrs: grau=12, aparições=1

✅ Seleção por grau (co-direções) concluída. Próximo: construir grafo e calcular métricas.


In [ ]:
# Etapa 12: Construir grafo NetworkX e calcular métricas (Grafo de Co-Direção)

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_directors = root / 'results' / 'grafo_diretores_codirecao'
out_dir_directors.mkdir(parents=True, exist_ok=True)

print(f"Nomes no top (por grau): {len(top_names_directors)}; pares filtrados: {len(filtered_pairs_directors)}")

# Construir grafo ponderado (undirected)
G_directors = nx.Graph()
G_directors.add_nodes_from(top_names_directors)
for (d1, d2), w in filtered_pairs_directors.items():
    # adicionar aresta com atributo weight
    G_directors.add_edge(d1, d2, weight=w)

n_nodes_dir = G_directors.number_of_nodes()
n_edges_dir = G_directors.number_of_edges()
print(f"Grafo: nós={n_nodes_dir}, arestas={n_edges_dir}")

# Métricas globais
density_dir = nx.density(G_directors)
components_dir = list(nx.connected_components(G_directors))
num_components_dir = len(components_dir)
largest_cc_dir = max(components_dir, key=len) if components_dir else set()
largest_cc_size_dir = len(largest_cc_dir)

print(f"Densidade: {density_dir:.6g}")
print(f"Componentes: {num_components_dir}; maior componente: {largest_cc_size_dir} nós")

# Centralidades
print('\nCalculando centralidades... (isso pode levar algum tempo para betweenness se exato)')
# Degree centrality (normalizada)
degree_c_dir = nx.degree_centrality(G_directors)
# Betweenness centrality (exata ou aproximada)
betweenness_c_dir = nx.betweenness_centrality(G_directors)
# Closeness centrality
closeness_c_dir = nx.closeness_centrality(G_directors)
# PageRank (usa weights)
pagerank_dir = nx.pagerank(G_directors, alpha=0.85, max_iter=100)

# Degree raw (número de vizinhos)
degree_raw_dir = dict(G_directors.degree())

# Montar DataFrame com métricas
metrics_df_dir = pd.DataFrame({
    'Director': list(G_directors.nodes()),
    'Degree': [degree_raw_dir.get(n, 0) for n in G_directors.nodes()],
    'DegreeCentrality': [degree_c_dir.get(n, 0) for n in G_directors.nodes()],
    'Betweenness': [betweenness_c_dir.get(n, 0) for n in G_directors.nodes()],
    'Closeness': [closeness_c_dir.get(n, 0) for n in G_directors.nodes()],
    'PageRank': [pagerank_dir.get(n, 0) for n in G_directors.nodes()],
})

# Normalizar colunas (opcional) — apenas como colunas separadas para inspeção
metrics_df_dir['Degree_norm'] = (metrics_df_dir['Degree'] - metrics_df_dir['Degree'].min()) / (metrics_df_dir['Degree'].max() - metrics_df_dir['Degree'].min())
metrics_df_dir['Betweenness_norm'] = (metrics_df_dir['Betweenness'] - metrics_df_dir['Betweenness'].min()) / (metrics_df_dir['Betweenness'].max() - metrics_df_dir['Betweenness'].min())
metrics_df_dir['Closeness_norm'] = (metrics_df_dir['Closeness'] - metrics_df_dir['Closeness'].min()) / (metrics_df_dir['Closeness'].max() - metrics_df_dir['Closeness'].min())
metrics_df_dir['PageRank_norm'] = (metrics_df_dir['PageRank'] - metrics_df_dir['PageRank'].min()) / (metrics_df_dir['PageRank'].max() - metrics_df_dir['PageRank'].min())

# Salvar resultados
metrics_csv_dir = out_dir_directors / 'graph_metrics.csv'
metrics_df_dir.to_csv(metrics_csv_dir, index=False)

# Mostrar top 10 por cada medida
print(f"\nMétricas salvas em: {metrics_csv_dir}")

print('\nTop 10 por Degree:')
print(metrics_df_dir.sort_values('Degree', ascending=False).head(10)[['Director','Degree']])
print('\nTop 10 por Betweenness:')
print(metrics_df_dir.sort_values('Betweenness', ascending=False).head(10)[['Director','Betweenness']])
print('\nTop 10 por Closeness:')
print(metrics_df_dir.sort_values('Closeness', ascending=False).head(10)[['Director','Closeness']])
print('\nTop 10 por PageRank:')
print(metrics_df_dir.sort_values('PageRank', ascending=False).head(10)[['Director','PageRank']])

print('\n✅ Etapa 12 concluída — métricas calculadas para o grafo de co-direção.')

Nomes no top (por grau): 100; pares filtrados: 495
Grafo: nós=100, arestas=495
Densidade: 0.1
Componentes: 8; maior componente: 22 nós

Calculando centralidades... (isso pode levar algum tempo para betweenness se exato)

Métricas salvas em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_codirecao/graph_metrics.csv

Top 10 por Degree:
            Director  Degree
0       Roger Allers      21
1         Chris Buck      12
2       Byron Howard      12
4     Shinji Aramaki      12
14      Mike Gabriel      12
6       Jennifer Lee      12
7    Patrick Osborne      12
8   Lauren MacMullan      12
10      Nathan Greno      12
9         John Kahrs      12

Top 10 por Betweenness:
            Director  Betweenness
4     Shinji Aramaki     0.022676
0       Roger Allers     0.022263
38  Masaru Matsumoto     0.022263
2       Byron Howard     0.000000
3     Peter Farrelly     0.000000
5     Rusty Cundieff     0.000000
6    

In [ ]:
# Etapa 13: Louvain + Geração de arquivos para Flourish (Grafo de Co-Direção)

# Parâmetros ajustáveis
LOUVAIN_RESOLUTION_DIR = 1
USE_LARGEST_CC_DIR = True
LOUVAIN_SEED_DIR = 42

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_dir = root / 'results' / 'grafo_diretores_codirecao'
out_dir_dir.mkdir(parents=True, exist_ok=True)

# Carregar grafo se necessário
try:
    G_directors
    filtered_pairs_directors
except NameError:
    raise FileNotFoundError('Grafo não encontrado. Execute a célula que constrói o grafo (Etapa 12) primeiro.')

# Subgrafo onde rodar Louvain
if USE_LARGEST_CC_DIR:
    comps_dir = list(nx.connected_components(G_directors))
    largest_dir = max(comps_dir, key=len)
    G_louvain_dir = G_directors.subgraph(largest_dir).copy()
else: 
    G_louvain_dir = G_directors

print(f"Rodando Louvain em n={G_louvain_dir.number_of_nodes()} nós, m={G_louvain_dir.number_of_edges()} arestas")
print(f"Parâmetro de resolução: {LOUVAIN_RESOLUTION_DIR}, seed: {LOUVAIN_SEED_DIR}")

# Calcular densidade do subgrafo Louvain
density_louvain_dir = nx.density(G_louvain_dir)
print(f"Densidade do grafo Louvain: {density_louvain_dir:.6f}")

# Executar Louvain
communities_sets_dir = louvain_communities(G_louvain_dir, weight='weight', resolution=LOUVAIN_RESOLUTION_DIR, seed=LOUVAIN_SEED_DIR)
communities_list_dir = [list(c) for c in communities_sets_dir]

# Calcular modularidade
mod_dir = modularity(G_louvain_dir, communities_list_dir, weight='weight')
num_communities_dir = len(communities_list_dir)

print(f"\nModularidade obtida: {mod_dir:.3f}")
print(f"Número de comunidades: {num_communities_dir}")

# Estatísticas das comunidades
sizes_dir = pd.Series([len(c) for c in communities_list_dir])
print(f"\nDistribuição de tamanhos (min={sizes_dir.min()}, max={sizes_dir.max()}, mean={sizes_dir.mean():.1f}, median={sizes_dir.median():.1f})")

# Criar dicionário node -> community_id
partition_dir = {}
for cid, comm in enumerate(communities_list_dir):
    for node in comm:
        partition_dir[node] = cid

# Estender partição para todos os nós
comm_dict_louvain_dir = {}
for node in G_directors.nodes():
    if node in partition_dir:
        comm_dict_louvain_dir[node] = partition_dir[node]
    else:
        comm_dict_louvain_dir[node] = -1

print("\n" + "="*80)
print("GERANDO ARQUIVOS PARA FLOURISH (DIRETORES - CO-DIREÇÃO)")
print("="*80)

# ========== 1. GERAR links.csv ==========
links_data_dir = []
for (d1, d2), weight in filtered_pairs_directors.items():
    if d1 in G_louvain_dir.nodes() and d2 in G_louvain_dir.nodes():
        links_data_dir.append((d1, d2, weight))

links_df_dir = pd.DataFrame(links_data_dir, columns=['Source', 'Target', 'Value'])
links_csv_dir = out_dir_dir / 'links.csv'
links_df_dir.to_csv(links_csv_dir, index=False)

print(f"\n✅ links.csv gerado:")
print(f"   - {len(links_df_dir)} arestas")
print(f"   - Arquivo: {links_csv_dir}")

# ========== 2. GERAR points.csv ==========
# Calcular métricas de centralidade para o subgrafo Louvain
betweenness_louvain_dir = nx.betweenness_centrality(G_louvain_dir, weight='weight')
closeness_louvain_dir = nx.closeness_centrality(G_louvain_dir)
pagerank_louvain_dir = nx.pagerank(G_louvain_dir, weight='weight')

# Normalizar métricas (min-max normalization)
betweenness_values_dir = list(betweenness_louvain_dir.values())
closeness_values_dir = list(closeness_louvain_dir.values())
pagerank_values_dir = list(pagerank_louvain_dir.values())

betw_min_dir, betw_max_dir = min(betweenness_values_dir), max(betweenness_values_dir)
clos_min_dir, clos_max_dir = min(closeness_values_dir), max(closeness_values_dir)
pr_min_dir, pr_max_dir = min(pagerank_values_dir), max(pagerank_values_dir)

points_data_dir = []
for director in G_louvain_dir.nodes():
    # Normalizar valores
    betw_norm_dir = (betweenness_louvain_dir[director] - betw_min_dir) / (betw_max_dir - betw_min_dir) if betw_max_dir > betw_min_dir else 0
    clos_norm_dir = (closeness_louvain_dir[director] - clos_min_dir) / (clos_max_dir - clos_min_dir) if clos_max_dir > clos_min_dir else 0
    pr_norm_dir = (pagerank_louvain_dir[director] - pr_min_dir) / (pr_max_dir - pr_min_dir) if pr_max_dir > pr_min_dir else 0
    
    points_data_dir.append((
        director,  # id
        comm_dict_louvain_dir[director],  # Community
        G_louvain_dir.degree(director),  # Degree
        director_appearances.get(director, 0),  # Appearances
        round(betw_norm_dir, 4),  # Betweenness (normalizado)
        round(clos_norm_dir, 4),  # Closeness (normalizado)
        round(pr_norm_dir, 4)  # PageRank (normalizado)
    ))

points_df_dir = pd.DataFrame(points_data_dir, columns=['id', 'Community', 'Degree', 'Appearances', 'Betweenness', 'Closeness', 'PageRank'])
points_csv_dir = out_dir_dir / 'points.csv'
points_df_dir.to_csv(points_csv_dir, index=False)

print(f"\n✅ points.csv gerado:")
print(f"   - {len(points_df_dir)} diretores")
print(f"   - {points_df_dir['Community'].nunique()} comunidades")
print(f"   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank")
print(f"   - Arquivo: {points_csv_dir}")

# ========== ANÁLISE: TOP 10 POR CENTRALIDADE ==========
print("\n" + "="*80)
print("TOP 10 DIRETORES POR MEDIDA DE CENTRALIDADE (CO-DIREÇÃO)")
print("="*80)

print("\n🔗 Top 10 por Degree (número de conexões):")
for idx, row in points_df_dir.nlargest(10, 'Degree').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Degree']} conexões (filmes: {row['Appearances']})")

print("\n🌉 Top 10 por Betweenness (intermediação):")
for idx, row in points_df_dir.nlargest(10, 'Betweenness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Betweenness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n📍 Top 10 por Closeness (proximidade):")
for idx, row in points_df_dir.nlargest(10, 'Closeness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Closeness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n⭐ Top 10 por PageRank (importância):")
for idx, row in points_df_dir.nlargest(10, 'PageRank').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['PageRank']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n\n >>> Densidade do grafo de co-direção analisado: {:.6g} <<<".format(density_louvain_dir))

# ========== 3. SALVAR METADADOS ==========
louvain_stats_dir = {
    'resolution': LOUVAIN_RESOLUTION_DIR,
    'seed': LOUVAIN_SEED_DIR,
    'modularity': mod_dir,
    'num_communities': num_communities_dir,
    'nodes_analyzed': G_louvain_dir.number_of_nodes(),
    'edges_analyzed': G_louvain_dir.number_of_edges()
}

pd.DataFrame([louvain_stats_dir]).to_csv(out_dir_dir / 'louvain_stats.csv', index=False)

with open(out_dir_dir / 'communities_louvain.pkl', 'wb') as f:
    pickle.dump(comm_dict_louvain_dir, f)

# ========== 4. RESUMO DAS COMUNIDADES ==========
print("\n" + "="*80)
print("RESUMO DAS COMUNIDADES")
print("="*80)

comm_sizes_dir = points_df_dir['Community'].value_counts().sort_values(ascending=False)
print(f"\nTop 10 maiores comunidades (ID: tamanho):")
for idx, (comm_id, size) in enumerate(comm_sizes_dir.head(10).items(), 1):
    print(f"  {idx}. Comunidade {comm_id}: {size} diretores")

print("\n" + "="*80)
print("ARQUIVOS FLOURISH PRONTOS!")
print("="*80)
print(f"\n📁 Diretório: {out_dir_dir}")
print(f"   ├── links.csv    ({len(links_df_dir)} arestas)")
print(f"   ├── points.csv   ({len(points_df_dir)} nós, {num_communities_dir} comunidades)")
print(f"   ├── louvain_stats.csv")
print(f"   └── communities_louvain.pkl")
print("\n✨ Importe links.csv e points.csv no Flourish Network Graph!")

Rodando Louvain em n=22 nós, m=123 arestas
Parâmetro de resolução: 1, seed: 42
Densidade do grafo Louvain: 0.532468

Modularidade obtida: 0.388
Número de comunidades: 2

Distribuição de tamanhos (min=10, max=12, mean=11.0, median=11.0)

GERANDO ARQUIVOS PARA FLOURISH (DIRETORES - CO-DIREÇÃO)

✅ links.csv gerado:
   - 123 arestas
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_codirecao/links.csv

✅ points.csv gerado:
   - 22 diretores
   - 2 comunidades
   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_codirecao/points.csv

TOP 10 DIRETORES POR MEDIDA DE CENTRALIDADE (CO-DIREÇÃO)

🔗 Top 10 por Degree (número de conexões):
   15. Roger Allers: 21 conexões (filmes: 3)
   1. Mike Gabriel: 12 conexões (filmes: 1)
   2. Patrick Osborn

In [ ]:
# Etapa 14: Selecionar top diretores por aparições (filmes)

from collections import Counter

TOP_N_DIRECTORS_APPEARANCES = 100

# Ordenar diretores por número de aparições
director_ranking_appearances = sorted(director_appearances.items(), key=lambda x: x[1], reverse=True)

# Pegar top N
top_directors_appearances = director_ranking_appearances[:TOP_N_DIRECTORS_APPEARANCES]
top_names_directors_appearances = {director for director, _ in top_directors_appearances}

print(f"\n🎬 Top {TOP_N_DIRECTORS_APPEARANCES} diretores por número de filmes:")
print(f"   - Diretor com mais filmes: {top_directors_appearances[0][0]} ({top_directors_appearances[0][1]} filmes)")
print(f"   - Diretor na posição {TOP_N_DIRECTORS_APPEARANCES}: {top_directors_appearances[-1][0]} ({top_directors_appearances[-1][1]} filmes)")

# Filtrar pares de co-direção para incluir apenas top diretores
filtered_pairs_directors_appearances = {}
for (d1, d2), weight in pair_counts_directors.items():
    if d1 in top_names_directors_appearances and d2 in top_names_directors_appearances:
        filtered_pairs_directors_appearances[(d1, d2)] = weight

print(f"\n📊 Arestas após filtro:")
print(f"   - Total de arestas (co-direções): {len(filtered_pairs_directors_appearances)}")

# Calcular grau para os top diretores
director_degrees_appearances = Counter()
for (d1, d2) in filtered_pairs_directors_appearances.keys():
    director_degrees_appearances[d1] += 1
    director_degrees_appearances[d2] += 1

print(f"\n🔗 Distribuição de grau (conexões):")
degrees_series_app = pd.Series(list(director_degrees_appearances.values()))
print(f"   - Média: {degrees_series_app.mean():.2f}")
print(f"   - Mediana: {degrees_series_app.median():.1f}")
print(f"   - Máximo: {degrees_series_app.max()}")
print(f"   - Mínimo: {degrees_series_app.min()}")


🎬 Top 100 diretores por número de filmes:
   - Diretor com mais filmes: Rajiv Chilaka (22 filmes)
   - Diretor na posição 100: Mahesh Manjrekar (5 filmes)

📊 Arestas após filtro:
   - Total de arestas (co-direções): 9

🔗 Distribuição de grau (conexões):
   - Média: 1.80
   - Mediana: 1.0
   - Máximo: 3
   - Mínimo: 1


In [ ]:
# Etapa 15: Construir grafo NetworkX e calcular métricas (Diretores com Mais Filmes)

import networkx as nx
import pandas as pd
from pathlib import Path

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_app_directors = root / 'results' / 'grafo_diretores_mais_filmes'
out_dir_app_directors.mkdir(parents=True, exist_ok=True)

# Criar grafo
G_directors_appearances = nx.Graph()

# Adicionar arestas com peso
for (d1, d2), weight in filtered_pairs_directors_appearances.items():
    G_directors_appearances.add_edge(d1, d2, weight=weight)

# Adicionar atributo de aparições aos nós
for director in G_directors_appearances.nodes():
    G_directors_appearances.nodes[director]['appearances'] = director_appearances.get(director, 0)

print(f"\n📊 GRAFO CONSTRUÍDO (Diretores com Mais Filmes)")
print(f"   - Nós: {G_directors_appearances.number_of_nodes()}")
print(f"   - Arestas: {G_directors_appearances.number_of_edges()}")
print(f"   - Densidade: {nx.density(G_directors_appearances):.4f}")

# Componentes conexas
num_cc_app_dir = nx.number_connected_components(G_directors_appearances)
cc_sizes_app_dir = [len(c) for c in nx.connected_components(G_directors_appearances)]

print(f"\n🔗 Componentes Conexas:")
print(f"   - Total: {num_cc_app_dir}")
print(f"   - Maior componente: {max(cc_sizes_app_dir)} nós ({max(cc_sizes_app_dir)/G_directors_appearances.number_of_nodes()*100:.1f}%)")

# Calcular métricas de centralidade
print("\n⏳ Calculando métricas de centralidade...")

degree_centrality_app_dir = nx.degree_centrality(G_directors_appearances)
betweenness_centrality_app_dir = nx.betweenness_centrality(G_directors_appearances, weight='weight')
closeness_centrality_app_dir = nx.closeness_centrality(G_directors_appearances)
pagerank_app_dir = nx.pagerank(G_directors_appearances, weight='weight')

print("✅ Métricas calculadas!")

# Criar DataFrame com todas as métricas
metrics_data_app_dir = []
for director in G_directors_appearances.nodes():
    metrics_data_app_dir.append({
        'director': director,
        'degree': G_directors_appearances.degree(director),
        'degree_centrality': degree_centrality_app_dir[director],
        'betweenness_centrality': betweenness_centrality_app_dir[director],
        'closeness_centrality': closeness_centrality_app_dir[director],
        'pagerank': pagerank_app_dir[director],
        'appearances': director_appearances.get(director, 0)
    })

metrics_df_app_dir = pd.DataFrame(metrics_data_app_dir)

# Ordenar por PageRank
metrics_df_app_dir = metrics_df_app_dir.sort_values('pagerank', ascending=False)

# Salvar métricas
metrics_csv_app_dir = out_dir_app_directors / 'graph_metrics.csv'
metrics_df_app_dir.to_csv(metrics_csv_app_dir, index=False)

print(f"\n💾 Métricas salvas em: {metrics_csv_app_dir}")

# Exibir top 10 por PageRank
print(f"\n🏆 Top 10 diretores por PageRank:")
for idx, row in metrics_df_app_dir.head(10).iterrows():
    print(f"   {row.name+1}. {row['director']}: {row['pagerank']:.4f} (filmes: {int(row['appearances'])}, grau: {int(row['degree'])})")


📊 GRAFO CONSTRUÍDO (Diretores com Mais Filmes)
   - Nós: 10
   - Arestas: 9
   - Densidade: 0.2000

🔗 Componentes Conexas:
   - Total: 4
   - Maior componente: 4 nós (40.0%)

⏳ Calculando métricas de centralidade...
✅ Métricas calculadas!

💾 Métricas salvas em: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_mais_filmes/graph_metrics.csv

🏆 Top 10 diretores por PageRank:
   1. Alex Woo: 0.1000 (filmes: 5, grau: 1)
   2. Stanley Moore: 0.1000 (filmes: 5, grau: 1)
   3. Priyadarshan: 0.1000 (filmes: 7, grau: 1)
   4. Rathindran R Prasad: 0.1000 (filmes: 5, grau: 1)
   5. Anurag Kashyap: 0.1000 (filmes: 9, grau: 3)
   6. Dibakar Banerjee: 0.1000 (filmes: 7, grau: 3)
   7. Karan Johar: 0.1000 (filmes: 6, grau: 3)
   8. Zoya Akhtar: 0.1000 (filmes: 6, grau: 3)
   9. Jan Suter: 0.1000 (filmes: 21, grau: 1)
   10. Raúl Campos: 0.1000 (filmes: 19, grau: 1)


In [ ]:
# Etapa 16: Louvain + Geração de arquivos para Flourish + Análise de Comunidades (Diretores com Mais Filmes)

# Parâmetros ajustáveis
LOUVAIN_RESOLUTION_DIR_APP = 1
USE_LARGEST_CC_DIR_APP = True
LOUVAIN_SEED_DIR_APP = 42

root = Path().resolve().parent if (Path.cwd().name == 'graphs') else Path().resolve()
out_dir_dir_app = root / 'results' / 'grafo_diretores_mais_filmes'
out_dir_dir_app.mkdir(parents=True, exist_ok=True)

# Carregar grafo se necessário
try:
    G_directors_appearances
    filtered_pairs_directors_appearances
except NameError:
    raise FileNotFoundError('Grafo não encontrado. Execute a célula que constrói o grafo (Etapa 15) primeiro.')

# Subgrafo onde rodar Louvain
if USE_LARGEST_CC_DIR_APP:
    comps_dir_app = list(nx.connected_components(G_directors_appearances))
    largest_dir_app = max(comps_dir_app, key=len)
    G_louvain_dir_app = G_directors_appearances.subgraph(largest_dir_app).copy()
else:
    G_louvain_dir_app = G_directors_appearances

print(f"Rodando Louvain em n={G_louvain_dir_app.number_of_nodes()} nós, m={G_louvain_dir_app.number_of_edges()} arestas")
print(f"Parâmetro de resolução: {LOUVAIN_RESOLUTION_DIR_APP}, seed: {LOUVAIN_SEED_DIR_APP}")

# Calcular densidade do subgrafo Louvain
density_louvain_dir_app = nx.density(G_louvain_dir_app)
print(f"Densidade do grafo Louvain: {density_louvain_dir_app:.6f}")

# Executar Louvain
communities_sets_dir_app = louvain_communities(G_louvain_dir_app, weight='weight', resolution=LOUVAIN_RESOLUTION_DIR_APP, seed=LOUVAIN_SEED_DIR_APP)
communities_list_dir_app = [list(c) for c in communities_sets_dir_app]

# Calcular modularidade
mod_dir_app = modularity(G_louvain_dir_app, communities_list_dir_app, weight='weight')
num_communities_dir_app = len(communities_list_dir_app)

print(f"\nModularidade obtida: {mod_dir_app:.3f}")
print(f"Número de comunidades: {num_communities_dir_app}")

# Estatísticas das comunidades
sizes_dir_app = pd.Series([len(c) for c in communities_list_dir_app])
print(f"\nDistribuição de tamanhos (min={sizes_dir_app.min()}, max={sizes_dir_app.max()}, mean={sizes_dir_app.mean():.1f}, median={sizes_dir_app.median():.1f})")

# Criar dicionário node -> community_id
partition_dir_app = {}
for cid, comm in enumerate(communities_list_dir_app):
    for node in comm:
        partition_dir_app[node] = cid

# Estender partição para todos os nós
comm_dict_louvain_dir_app = {}
for node in G_directors_appearances.nodes():
    if node in partition_dir_app:
        comm_dict_louvain_dir_app[node] = partition_dir_app[node]
    else:
        comm_dict_louvain_dir_app[node] = -1

print("\n" + "="*80)
print("GERANDO ARQUIVOS PARA FLOURISH (DIRETORES - MAIS FILMES)")
print("="*80)

# ========== 1. GERAR links.csv ==========
links_data_dir_app = []
for (d1, d2), weight in filtered_pairs_directors_appearances.items():
    if d1 in G_louvain_dir_app.nodes() and d2 in G_louvain_dir_app.nodes():
        links_data_dir_app.append((d1, d2, weight))

links_df_dir_app = pd.DataFrame(links_data_dir_app, columns=['Source', 'Target', 'Value'])
links_csv_dir_app = out_dir_dir_app / 'links.csv'
links_df_dir_app.to_csv(links_csv_dir_app, index=False)

print(f"\n✅ links.csv gerado:")
print(f"   - {len(links_df_dir_app)} arestas")
print(f"   - Arquivo: {links_csv_dir_app}")

# ========== 2. GERAR points.csv ==========
# Calcular métricas de centralidade para o subgrafo Louvain
betweenness_louvain_dir_app = nx.betweenness_centrality(G_louvain_dir_app, weight='weight')
closeness_louvain_dir_app = nx.closeness_centrality(G_louvain_dir_app)
pagerank_louvain_dir_app = nx.pagerank(G_louvain_dir_app, weight='weight')

# Normalizar métricas (min-max normalization)
betweenness_values_dir_app = list(betweenness_louvain_dir_app.values())
closeness_values_dir_app = list(closeness_louvain_dir_app.values())
pagerank_values_dir_app = list(pagerank_louvain_dir_app.values())

betw_min_dir_app, betw_max_dir_app = min(betweenness_values_dir_app), max(betweenness_values_dir_app)
clos_min_dir_app, clos_max_dir_app = min(closeness_values_dir_app), max(closeness_values_dir_app)
pr_min_dir_app, pr_max_dir_app = min(pagerank_values_dir_app), max(pagerank_values_dir_app)

points_data_dir_app = []
for director in G_louvain_dir_app.nodes():
    # Normalizar valores
    betw_norm_dir_app = (betweenness_louvain_dir_app[director] - betw_min_dir_app) / (betw_max_dir_app - betw_min_dir_app) if betw_max_dir_app > betw_min_dir_app else 0
    clos_norm_dir_app = (closeness_louvain_dir_app[director] - clos_min_dir_app) / (clos_max_dir_app - clos_min_dir_app) if clos_max_dir_app > clos_min_dir_app else 0
    pr_norm_dir_app = (pagerank_louvain_dir_app[director] - pr_min_dir_app) / (pr_max_dir_app - pr_min_dir_app) if pr_max_dir_app > pr_min_dir_app else 0
    
    points_data_dir_app.append((
        director,  # id
        comm_dict_louvain_dir_app[director],  # Community
        G_louvain_dir_app.degree(director),  # Degree
        director_appearances.get(director, 0),  # Appearances
        round(betw_norm_dir_app, 4),  # Betweenness (normalizado)
        round(clos_norm_dir_app, 4),  # Closeness (normalizado)
        round(pr_norm_dir_app, 4)  # PageRank (normalizado)
    ))

points_df_dir_app = pd.DataFrame(points_data_dir_app, columns=['id', 'Community', 'Degree', 'Appearances', 'Betweenness', 'Closeness', 'PageRank'])
points_csv_dir_app = out_dir_dir_app / 'points.csv'
points_df_dir_app.to_csv(points_csv_dir_app, index=False)

print(f"\n✅ points.csv gerado:")
print(f"   - {len(points_df_dir_app)} diretores")
print(f"   - {points_df_dir_app['Community'].nunique()} comunidades")
print(f"   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank")
print(f"   - Arquivo: {points_csv_dir_app}")

# ========== ANÁLISE: TOP 10 POR CENTRALIDADE ==========
print("\n" + "="*80)
print("TOP 10 DIRETORES POR MEDIDA DE CENTRALIDADE (MAIS FILMES)")
print("="*80)

print("\n🔗 Top 10 por Degree (número de conexões):")
for idx, row in points_df_dir_app.nlargest(10, 'Degree').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Degree']} conexões (filmes: {row['Appearances']})")

print("\n🌉 Top 10 por Betweenness (intermediação):")
for idx, row in points_df_dir_app.nlargest(10, 'Betweenness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Betweenness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n📍 Top 10 por Closeness (proximidade):")
for idx, row in points_df_dir_app.nlargest(10, 'Closeness').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['Closeness']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n⭐ Top 10 por PageRank (importância):")
for idx, row in points_df_dir_app.nlargest(10, 'PageRank').iterrows():
    print(f"   {idx+1}. {row['id']}: {row['PageRank']:.4f} (grau: {row['Degree']}, filmes: {row['Appearances']})")

print("\n\n >>> Densidade do grafo de diretores com mais filmes analisado: {:.6g} <<<".format(density_louvain_dir_app))

# ========== 3. SALVAR METADADOS ==========
louvain_stats_dir_app = {
    'resolution': LOUVAIN_RESOLUTION_DIR_APP,
    'seed': LOUVAIN_SEED_DIR_APP,
    'modularity': mod_dir_app,
    'num_communities': num_communities_dir_app,
    'nodes_analyzed': G_louvain_dir_app.number_of_nodes(),
    'edges_analyzed': G_louvain_dir_app.number_of_edges()
}

pd.DataFrame([louvain_stats_dir_app]).to_csv(out_dir_dir_app / 'louvain_stats.csv', index=False)

with open(out_dir_dir_app / 'communities_louvain.pkl', 'wb') as f:
    pickle.dump(comm_dict_louvain_dir_app, f)

# ========== 4. ANÁLISE DETALHADA DAS COMUNIDADES ==========
print("\n" + "="*80)
print("ANÁLISE DETALHADA DAS COMUNIDADES")
print("="*80)

community_analysis_dir_app = []
for comm_id, members in enumerate(communities_list_dir_app):
    # Filtrar apenas diretores nesta comunidade
    comm_df = points_df_dir_app[points_df_dir_app['Community'] == comm_id]
    
    # Calcular estatísticas
    total_appearances = comm_df['Appearances'].sum()
    avg_appearances = comm_df['Appearances'].mean()
    max_appearances = comm_df['Appearances'].max()
    
    # Diretor com mais filmes na comunidade
    top_director = comm_df.loc[comm_df['Appearances'].idxmax(), 'id']
    
    community_analysis_dir_app.append({
        'community_id': comm_id,
        'size': len(members),
        'total_appearances': total_appearances,
        'avg_appearances': avg_appearances,
        'max_appearances': max_appearances,
        'top_director': top_director
    })

community_df_dir_app = pd.DataFrame(community_analysis_dir_app)
community_df_dir_app = community_df_dir_app.sort_values('size', ascending=False)

# Salvar análise
community_csv_dir_app = out_dir_dir_app / 'community_analysis.csv'
community_df_dir_app.to_csv(community_csv_dir_app, index=False)

print(f"\n✅ Análise de comunidades salva em: {community_csv_dir_app}")

# Exibir top 5 comunidades
print(f"\n🏆 Top 5 maiores comunidades:")
for idx, row in community_df_dir_app.head(5).iterrows():
    print(f"\n   Comunidade {int(row['community_id'])}:")
    print(f"   - Tamanho: {int(row['size'])} diretores")
    print(f"   - Total de filmes: {int(row['total_appearances'])}")
    print(f"   - Média de filmes: {row['avg_appearances']:.1f}")
    print(f"   - Diretor mais produtivo: {row['top_director']} ({int(row['max_appearances'])} filmes)")

print("\n" + "="*80)
print("ARQUIVOS FLOURISH PRONTOS!")
print("="*80)
print(f"\n📁 Diretório: {out_dir_dir_app}")
print(f"   ├── links.csv              ({len(links_df_dir_app)} arestas)")
print(f"   ├── points.csv             ({len(points_df_dir_app)} nós, {num_communities_dir_app} comunidades)")
print(f"   ├── community_analysis.csv ({num_communities_dir_app} comunidades)")
print(f"   ├── louvain_stats.csv")
print(f"   └── communities_louvain.pkl")
print("\n✨ Importe links.csv e points.csv no Flourish Network Graph!")

Rodando Louvain em n=4 nós, m=6 arestas
Parâmetro de resolução: 1, seed: 42
Densidade do grafo Louvain: 1.000000

Modularidade obtida: 0.000
Número de comunidades: 1

Distribuição de tamanhos (min=4, max=4, mean=4.0, median=4.0)

GERANDO ARQUIVOS PARA FLOURISH (DIRETORES - MAIS FILMES)

✅ links.csv gerado:
   - 6 arestas
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_mais_filmes/links.csv

✅ points.csv gerado:
   - 4 diretores
   - 1 comunidades
   - Colunas: id, Community, Degree, Appearances, Betweenness, Closeness, PageRank
   - Arquivo: /home/afmireski/Documentos/BCC/p8/analise_de_redes_sociais_utilizando_grafos/projeto/opt108_projeto/results/grafo_diretores_mais_filmes/points.csv

TOP 10 DIRETORES POR MEDIDA DE CENTRALIDADE (MAIS FILMES)

🔗 Top 10 por Degree (número de conexões):
   1. Karan Johar: 3 conexões (filmes: 6)
   2. Dibakar Banerjee: 3 conexões (filmes: 7)
   3. Zoya Akhtar: 3 con